In [22]:
import tushare as ts
import pandas as pd
import numpy as np
import baostock as bs
import datetime
import os
import talib

## 获取沪深300数据

1. 沪深300
2. 2024-1-1 ~
3. 价格信息：开盘价、收盘价、最高价、最?低价、涨停价、跌停价
4. 价格波动率
5. 技术指标：趋势指标、动量指标、成交量指标

In [4]:
# 设置tushare的token
ts.set_token('b7b8bfa3fa50b278dd71aee03bbe33f283b2a076d0b75b4ef73c046c')
pro = ts.pro_api()

In [5]:
# 获取沪深300成分股列表
index_weight_df = pro.index_weight(index_code='000300.SH', start_date='20240101', end_date='20250322')
stock_list = index_weight_df['con_code'].tolist()

Exception: 抱歉，您没有接口访问权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

# 分隔符

# 上面那个数据看起来挺靠谱，长期维护，但他妈的要收钱

In [7]:
# 登录系统
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 获取沪深300成分股
rs = bs.query_hs300_stocks()
if rs.error_code != '0':
    print(f'查询沪深300成分股失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
else:
    # 保存成DataFrame
    hs300_stocks = []
    while rs.next():
        hs300_stocks.append(rs.get_row_data())
    df_hs300 = pd.DataFrame(hs300_stocks, columns=rs.fields)
    print(df_hs300)

# 登出系统
bs.logout()

login success!
登录成功
     updateDate       code code_name
0    2025-03-17  sh.600000      浦发银行
1    2025-03-17  sh.600009      上海机场
2    2025-03-17  sh.600010      包钢股份
3    2025-03-17  sh.600011      华能国际
4    2025-03-17  sh.600015      华夏银行
..          ...        ...       ...
295  2025-03-17  sz.300832       新产业
296  2025-03-17  sz.300896       爱美客
297  2025-03-17  sz.300979      华利集团
298  2025-03-17  sz.300999       金龙鱼
299  2025-03-17  sz.301269      华大九天

[300 rows x 3 columns]
logout success!


## 简单查看HS300

In [12]:
df_hs300

,updateDate,code,code_name
0,2025-03-17,sh.600000,浦发银行
1,2025-03-17,sh.600009,上海机场
2,2025-03-17,sh.600010,包钢股份
3,2025-03-17,sh.600011,华能国际
4,2025-03-17,sh.600015,华夏银行
...,...,...,...
295,2025-03-17,sz.300832,新产业
296,2025-03-17,sz.300896,爱美客
297,2025-03-17,sz.300979,华利集团
298,2025-03-17,sz.300999,金龙鱼


In [14]:
df_hs300.to_csv(os.path.join("/home/kennys/MineX/QuantTrading/dataset", "沪深300成分股.csv"))

## HS300查询

1. 设定起止事件查询
2. 获取所有成分股的信息

In [24]:
# 登录系统
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2025-02-01'
end_date = '2025-02-28'

# 初始化存储数据的列表
all_stock_data = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    all_stock_data.append(df_stock)

# 合并所有股票的数据
df_all_stocks = pd.concat(all_stock_data, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [25]:
df_all_stocks

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2025-02-05,sh.600000,10.6600,10.6900,10.4000,10.4100,10.6500,51723455,542382244.5300,0.176200
1,2025-02-06,sh.600000,10.3800,10.5000,10.3000,10.4300,10.4100,38598431,400818916.7100,0.131500
2,2025-02-07,sh.600000,10.4300,10.4500,10.3000,10.3400,10.4300,45600119,472723880.7700,0.155400
3,2025-02-10,sh.600000,10.3200,10.4400,10.2300,10.2700,10.3400,41234671,426297964.4000,0.140500
4,2025-02-11,sh.600000,10.2900,10.3800,10.2300,10.3000,10.2700,34651727,356843469.5200,0.118100
...,...,...,...,...,...,...,...,...,...,...
5395,2025-02-24,sz.301269,117.3100,119.4800,115.4800,117.7600,117.3100,6239560,732092575.6500,2.378600
5396,2025-02-25,sz.301269,115.8000,120.8300,114.4000,118.1100,117.7600,7596517,893618212.6600,2.895900
5397,2025-02-26,sz.301269,118.1800,118.5100,114.2800,117.4900,118.1100,7569108,879230969.9400,2.885400
5398,2025-02-27,sz.301269,116.9800,117.6300,112.2500,114.0900,117.4900,6267180,718273060.8000,2.389100


In [16]:
df_all_stocks.to_csv(os.path.join("/home/kennys/MineX/QuantTrading/dataset", "沪深300-2025年2月数据.csv"))

In [11]:
unique_stock_list = df_all_stocks["code"].unique()
len(unique_stock_list)

300

## 针对2025-2数据进行计算

1. 涨停价、跌停价

- Free数据都是日K数据，需要自己计算一些指标
- 非ST票涨跌幅度为10%，ST票为5%
- 沪深300的票应该不存在ST票

2. 计算价格波动率

- 计算对数收益率的标准差
- 年化波动率 252

3. 技术指标：由于只有2月份共18天的数据，计算10日均线时前10天是NAN，30日均线都是NAN

- 10日均线
- 30日均线


In [18]:
def calculate_limited_prices(row):
    preclose = float(row["preclose"])
    if row["isST"] == 1:
        limit_up = preclose * 1.05
        limit_down = preclose * 0.95
    else:
        limit_up = preclose * 1.10
        limit_down = preclose * 0.90
    
    return pd.Series([limit_up, limit_down])

In [26]:
"""HS300不存在ST"""
using_df = df_all_stocks.copy()
using_df["isST"] = "0"

In [27]:
using_df[['limit_up', 'limit_down']] = using_df.apply(calculate_limited_prices, axis=1)

In [28]:
using_df

,date,code,open,high,low,close,preclose,volume,amount,turn,isST,limit_up,limit_down
0,2025-02-05,sh.600000,10.6600,10.6900,10.4000,10.4100,10.6500,51723455,542382244.5300,0.176200,0,11.715,9.585
1,2025-02-06,sh.600000,10.3800,10.5000,10.3000,10.4300,10.4100,38598431,400818916.7100,0.131500,0,11.451,9.369
2,2025-02-07,sh.600000,10.4300,10.4500,10.3000,10.3400,10.4300,45600119,472723880.7700,0.155400,0,11.473,9.387
3,2025-02-10,sh.600000,10.3200,10.4400,10.2300,10.2700,10.3400,41234671,426297964.4000,0.140500,0,11.374,9.306
4,2025-02-11,sh.600000,10.2900,10.3800,10.2300,10.3000,10.2700,34651727,356843469.5200,0.118100,0,11.297,9.243
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,2025-02-24,sz.301269,117.3100,119.4800,115.4800,117.7600,117.3100,6239560,732092575.6500,2.378600,0,129.041,105.579
5396,2025-02-25,sz.301269,115.8000,120.8300,114.4000,118.1100,117.7600,7596517,893618212.6600,2.895900,0,129.536,105.984
5397,2025-02-26,sz.301269,118.1800,118.5100,114.2800,117.4900,118.1100,7569108,879230969.9400,2.885400,0,129.921,106.299
5398,2025-02-27,sz.301269,116.9800,117.6300,112.2500,114.0900,117.4900,6267180,718273060.8000,2.389100,0,129.239,105.741


In [29]:
"""价格波动率"""
using_df['log_return'] = np.log(using_df['close'].astype(float) / using_df['preclose'].astype(float))

# 计算每只股票的波动率
volatility = using_df.groupby('code')['log_return'].std() * np.sqrt(252)  # 年化波动率
df_volatility = volatility.reset_index()
df_volatility.columns = ['code', 'annual_volatility']

In [30]:
df_volatility

,code,annual_volatility
0,sh.600000,0.171293
1,sh.600009,0.101224
2,sh.600010,0.302658
3,sh.600011,0.157958
4,sh.600015,0.127131
...,...,...
295,sz.300832,0.373336
296,sz.300896,0.372197
297,sz.300979,0.344866
298,sz.300999,0.230561


In [31]:
df_volatility.to_csv(os.path.join("/home/kennys/MineX/QuantTrading/dataset", "沪深300-价格波动率.csv"))

In [33]:
"""计算技术指标"""
using_df = using_df.sort_values(by=['code', 'date'])

# 计算10日和30日移动平均线
using_df['SMA_10'] = using_df.groupby('code')['close'].transform(lambda x: talib.SMA(x.astype(float), timeperiod=10))
using_df['SMA_30'] = using_df.groupby('code')['close'].transform(lambda x: talib.SMA(x.astype(float), timeperiod=30))


In [34]:
using_df

,date,code,open,high,low,close,preclose,volume,amount,turn,isST,limit_up,limit_down,log_return,SMA_10,SMA_30
0,2025-02-05,sh.600000,10.6600,10.6900,10.4000,10.4100,10.6500,51723455,542382244.5300,0.176200,0,11.715,9.585,-0.022793,NaN,NaN
1,2025-02-06,sh.600000,10.3800,10.5000,10.3000,10.4300,10.4100,38598431,400818916.7100,0.131500,0,11.451,9.369,0.001919,NaN,NaN
2,2025-02-07,sh.600000,10.4300,10.4500,10.3000,10.3400,10.4300,45600119,472723880.7700,0.155400,0,11.473,9.387,-0.008666,NaN,NaN
3,2025-02-10,sh.600000,10.3200,10.4400,10.2300,10.2700,10.3400,41234671,426297964.4000,0.140500,0,11.374,9.306,-0.006793,NaN,NaN
4,2025-02-11,sh.600000,10.2900,10.3800,10.2300,10.3000,10.2700,34651727,356843469.5200,0.118100,0,11.297,9.243,0.002917,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,2025-02-24,sz.301269,117.3100,119.4800,115.4800,117.7600,117.3100,6239560,732092575.6500,2.378600,0,129.041,105.579,0.003829,115.323,NaN
5396,2025-02-25,sz.301269,115.8000,120.8300,114.4000,118.1100,117.7600,7596517,893618212.6600,2.895900,0,129.536,105.984,0.002968,115.434,NaN
5397,2025-02-26,sz.301269,118.1800,118.5100,114.2800,117.4900,118.1100,7569108,879230969.9400,2.885400,0,129.921,106.299,-0.005263,115.387,NaN
5398,2025-02-27,sz.301269,116.9800,117.6300,112.2500,114.0900,117.4900,6267180,718273060.8000,2.389100,0,129.239,105.741,-0.029366,115.496,NaN


In [35]:
using_df.to_csv(os.path.join("/home/kennys/MineX/QuantTrading/dataset", "沪深300-2025年2月计算十日三十日均线.csv"))

# 此处又是分割线

In [36]:
# 登录系统
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2024-01-01'
end_date = '2025-02-28'

# 初始化存储数据的列表
data = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    data.append(df_stock)

# 合并所有股票的数据
data_241_252 = pd.concat(data, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [37]:
data_241_252

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2024-01-02,sh.600000,6.6300,6.6500,6.6000,6.6000,6.6200,22066700,146066303.7200,0.075200
1,2024-01-03,sh.600000,6.5900,6.6500,6.5900,6.6400,6.6000,18203654,120639706.0100,0.062000
2,2024-01-04,sh.600000,6.6400,6.6700,6.5500,6.6200,6.6400,28885978,190580609.9900,0.098400
3,2024-01-05,sh.600000,6.6000,6.7600,6.5900,6.6800,6.6200,44421387,296976885.7900,0.151300
4,2024-01-08,sh.600000,6.6800,6.7100,6.5600,6.5900,6.6800,37520337,247977824.9800,0.127800
...,...,...,...,...,...,...,...,...,...,...
83395,2025-02-24,sz.301269,117.3100,119.4800,115.4800,117.7600,117.3100,6239560,732092575.6500,2.378600
83396,2025-02-25,sz.301269,115.8000,120.8300,114.4000,118.1100,117.7600,7596517,893618212.6600,2.895900
83397,2025-02-26,sz.301269,118.1800,118.5100,114.2800,117.4900,118.1100,7569108,879230969.9400,2.885400
83398,2025-02-27,sz.301269,116.9800,117.6300,112.2500,114.0900,117.4900,6267180,718273060.8000,2.389100


In [38]:
data_241_252.to_csv(os.path.join("/home/kennys/MineX/QuantTrading/dataset", "沪深300-2024年1月至2025年2月数据.csv"))